In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

from sklearn import datasets, linear_model, preprocessing
import numpy.polynomial.polynomial as poly

import lime
import lime.lime_tabular

import copy

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [18]:
df = pd.read_csv("working_data_raw.csv")
data_set = df.values
data_set = np.delete(data_set, 0, 1) # Deleting dataframe extra column

print(data_set[75][10])
print(data_set[75][0])

print(data_set[550][19])
print(data_set[550][0])

print(data_set[675][15])
print(data_set[550][0])

3.0
1.0
63.0
0.0
9.0
0.0


In [19]:
class ModelError(Exception):
    pass


def remove_col_with_vals(data, vals):
    no_cols = data.shape[1]
    no_rows = data.shape[0]
    row = 0
    while (no_rows > row):
        col = 0
        while (no_cols > col):
            if (data[row][col] in vals):
                data = np.delete(data, col, 1)
                no_cols -= 1
            else:
                col += 1
        row += 1     
    return data

def predict_values_lin_reg(X_tr,y_tr,X_test):
    model = linear_model.LinearRegression()
    model.fit(X_tr, y_tr)
    pred = model.predict(X_test)
    return pred

def data_spliter(all_data,target_col,target_val):
    y = all_data[:,target_col:target_col+1]
    X = np.delete(all_data,target_col,1)
    
    # Will hold the X for the y values that need to be predicted
    X_target = np.zeros((1,X.shape[1]))
    
    row_no = 0 
    for val in y:
        if (val[0] == target_val):
            X_target = np.append(X_target,X[row_no:row_no+1,:],axis=0)
            X = np.delete(X, row_no, 0)
            y = np.delete(y, row_no, 0) 
        else:
            row_no += 1
            
    X_target = np.delete(X_target,0,0)
    
    return X,y,X_target

def combine_parts_inorder(X,y,X_target,y_target,target_col):
    y_target = y_target.reshape((y_target.shape[0],1))
    y_full = np.append(y_target,y,axis=0)
    X_full = np.append(X_target,X,axis=0)
    
    data = np.append(X_full[:,:target_col],y_full,axis=1)
    data = np.append(data,X_full[:,target_col:],axis=1)
    return data

def process_and_predict(all_data,target_col,target_val,exclude=None,model="linear"):
    
    # -- Split data --
    X,y,X_target = data_spliter(all_data,target_col,target_val)
    
    # -- Remove certain columns --
    if (exclude != None):
        y_tr = np.copy(y)
        X_tr = remove_col_with_vals(X,exclude)
        X_pred = remove_col_with_vals(X_target,exclude) # The x used to predict
        X_tr = np.delete(X_tr, 0, 1) # Removes the target column to avoid leaking
        X_pred = np.delete(X_pred,0,1)
    else:
        y_tr = np.copy(y)
        X_tr = np.delete(X, 0, 1)
        X_pred = np.delete(X_target,0,1) 
        
    # -- Run regression --
    if (model == "linear"):
        y_target = predict_values_lin_reg(X_tr,y_tr,X_pred)
    
    elif (model == "polynomial"):
        pass
    
    elif (model == "special"):
        X_avg = average_each_feature(X_pred)
        pred = predict_values_lin_reg(X_tr,y_tr,X_avg)
        print(pred)
        return pred
    
    else:
        raise ModelError("Model currently not available")
        
    final_data = combine_parts_inorder(X,y,X_target,y_target,target_col)
    
    return final_data

In [20]:
# --- X including data --- 
X = np.copy(data_set)

# --- Finding the missing -9 values ---
X = process_and_predict(X,1,-9,[-7])

# --- Finding the missing -7 values ---
X = process_and_predict(X,9,-7,[-7])
X = process_and_predict(X,15,-7)

# --- Rounding the values to nearest whole value ---
X = np.around(X.astype(np.float),0)

# --- Write final result to file ---
new_df = pd.DataFrame(X)
new_df.to_csv("testing_new.csv")


3.0
1.0
63.0
0.0
9.0
0.0


In [ ]:
# --- Splitting again to training and target data (to avoid data leaking) --- 
y = data_set[:,:1]
X = data_set[:,1:]

# --- Finding the missing -9 values ---
X = process_and_predict(X,0,-9,[-7])

# --- Finding the missing -7 values ---
X = process_and_predict(X,8,-7,[-7])
X = process_and_predict(X,14,-7)

# --- Rounding the values to nearest whole value ---
X = np.around(X.astype(np.float),0)

# --- Creating one whole dataset ---
data_set = np.append(y,np.copy(X),axis=1)

# --- Write final result to file ---
new_df = pd.DataFrame(data_set)
new_df.to_csv("test2.csv")



In [3]:
df = pd.read_csv("backup.csv")
data_set = df.values
data_set = np.delete(data_set, 0, 1)

def shift_categorical(data):
    col_ten = data[:,10]
    np.place(col_ten, col_ten == 1, 100) # hold
    np.place(col_ten, col_ten == 6, 1)
    np.place(col_ten, col_ten == 5, 1)
    np.place(col_ten, col_ten == 4, 6)
    np.place(col_ten, col_ten == 3, 5)
    np.place(col_ten, col_ten == 2, 4)
    np.place(col_ten, col_ten == 100, 3)
    np.place(col_ten, col_ten == 0, 2)
    np.place(col_ten, col_ten == 8, 0)
    np.place(col_ten, col_ten == 9, 0)
    
    col_eleven = data[:,11]
    np.place(col_eleven, col_eleven == 1, 0)
    np.place(col_eleven, col_eleven == 9, 0)
    np.place(col_eleven, col_eleven == 7, 1)
    np.place(col_eleven, col_eleven == 8, 7)

    data[:,10] = col_ten
    data[:,11] = col_eleven
    
    return data

data_set = shift_categorical(data_set)
new_df = pd.DataFrame(data_set)
new_df.to_csv("working_data_full.csv")
